In [1]:
#!/usr/bin/python3
### get general libraries
import os
import sys

import pandas as pd

In [2]:
### get PDB credentials
cwd = os.getcwd()
# check where this is:
#cwd
sys.path.insert(1, cwd+"/Notebooks")
import setEnv
setEnv.SetEnv()

# import API and check authentication
import itkdb
myClient = itkdb.Client()
myClient.user.authenticate()
user=myClient.get('getUser', json={'userIdentity': myClient.user.identity})
user['firstName']

Done


'Kenneth'

In [3]:
# Check available component names and codes
instCode="GL"
projCode="P"

compTypeList=list(myClient.get('listComponents',json={'institution':instCode, 'project':projCode }))
compTypeMap=[{"name":w['componentType']['name'], "code":w['componentType']['code']} for w in compTypeList]
df_compType=pd.DataFrame(compTypeMap)
display(df_compType.drop_duplicates())

,name,code
0,Sensor Tile,SENSOR_TILE
11,Sensor Wafer,SENSOR_WAFER
378,Hybrid,HYBRID
391,Module,MODULE
393,Module carrier,MODULE_CARRIER


In [4]:
# Check available tests for componentType
compTypeCode="SENSOR_TILE"
compTestList=list(myClient.get('listTestTypes',json={'project':projCode, 'componentType':compTypeCode}))
compTestMap=[{"name":w['name'], "code":w['code']} for w in compTestList]
df_compTest=pd.DataFrame(compTestMap)
display(df_compTest.drop_duplicates())

,name,code
0,CV measurement,CV_MEASURE
1,IT measurement,IT_MEASURE
2,IV Measurment,IV_MEASURE
3,Inter-pixel resistance,INTER-PIXEL_RESISTANCE
4,Sensor Metrology: Sensor bow and thickness,SENSOR_METROLOGY
5,Visual inspection,VISUAL_INSPECTION


In [5]:
# make list of components of type (if status ready)
compTypeCode="SENSOR_TILE"
compList=list(myClient.get('listComponents',json={'institution':instCode, 'project':projCode, 'componentType':compTypeCode }))
compMap=[{"ASN":w['serialNumber'], "altID":w['alternativeIdentifier'], "code":w['code']} for w in compList if w['state']=="ready"]
df_comp=pd.DataFrame(compMap)
display(df_comp)

,ASN,altID,code
0,20UPGS85000174,V6-3804-11,42a3fcae3e210154f264bda2fbefcfbe
1,20UPGS85000173,V6-3804-10,ba4490f51ab4df63ce4be1ff78405805
2,20UPGS85000172,V6-3804-09,4d87bd194569e18bfb7cbc708e138e1a
3,20UPGS85300145,V6-3804-08,b39b3968b53b821dfd52a6c8fea65123
4,20UPGS85300144,V6-3804-07,0abd28800f00b88b9272cf3544fda55c
...,...,...,...
314,20UPGS85300002,V6-3795-05,5774afa3f39215efddb045e03a836a77
315,20UPGS85300001,V6-3795-04,86fc142c1966114cd0ec7a885186b767
316,20UPGS85000003,V6-3795-03,53d56890d3470f24d2d7631c4938c1e4
317,20UPGS85000002,V6-3795-02,b713290d9106ebf8dc5d05e6ec3402f1


In [8]:
# find tests of type for selected components
testType="CV_MEASURE"
for c in compMap:
    print(c['altID'])
    compObj=myClient.get('getComponent',json={'component':c['code']})
    for i in compObj['tests']:
        #print(i)
        if i['code']==testType:
            for r in i['testRuns']:
                print(r['id'])
                #if r['state']!="ready": continue
                try:
                    c[testType].append(r['id'])
                except KeyError:
                    c[testType]=[r['id']]

V6-3804-11
V6-3804-10
V6-3804-09
V6-3804-08
V6-3804-07
V6-3804-06
V6-3804-05
V6-3804-04
V6-3804-03
V6-3804-02
V6-3804-01
V6-3797-11
V6-3797-10
V6-3797-09
V6-3797-08
V6-3797-07
V6-3797-06
V6-3797-05
V6-3797-04
V6-3797-03
V6-3797-02
V6-3797-01
V6-3793-11
V6-3793-10
V6-3793-09
V6-3793-08
V6-3793-07
V6-3793-06
V6-3793-05
V6-3793-04
V6-3793-03
V6-3793-02
V6-3793-01
V6-3781-11
V6-3781-10
V6-3781-09
V6-3781-08
V6-3781-07
V6-3781-06
V6-3781-05
5fc7b922e268e5000a6f92ce
V6-3781-04
5fc7b612e268e5000a6f9274
5fc7b4e6e268e5000a6f9249
V6-3781-03
V6-3781-02
V6-3781-01
V6-3807-11
V6-3807-10
V6-3807-09
V6-3807-08
V6-3807-07
V6-3807-06
V6-3807-05
V6-3807-04
V6-3807-03
V6-3807-02
V6-3807-01
V6-3806-11
V6-3806-10
V6-3806-09
V6-3806-08
V6-3806-07
V6-3806-06
V6-3806-05
V6-3806-04
V6-3806-03
V6-3806-02
V6-3806-01
V6-3779-11
V6-3779-10
V6-3779-09
V6-3779-08
V6-3779-07
V6-3779-06
V6-3779-05
V6-3779-04
V6-3779-03
V6-3779-02
V6-3779-01
V6-3775-11
V6-3775-10
V6-3775-09
V6-3775-08
V6-3775-07
V6-3775-06
V6-3775-05
V

In [9]:
df_compAug=pd.DataFrame(compMap)
### nifty filter out NaN trick:
df_compAug.query(testType+" == "+testType)

,ASN,altID,code,CV_MEASURE
39,20UPGS85300127,V6-3781-05,cb5be8c42b77476dbb5c4a23757bb0ed,"[5fc7b922e268e5000a6f92ce, 5fc7b922e268e5000a6..."
40,20UPGS85300126,V6-3781-04,ed521d104b139cfc7b305e5547a2cb23,"[5fc7b612e268e5000a6f9274, 5fc7b4e6e268e5000a6..."


In [10]:
# edit tests
for c in compMap:
    try:
        for r in c[testType]:
            testObj=myClient.get('getTestRun',json={'testRun':r})
            for v in testObj['results']:
                ### edits here
                if v['code']=="V_FULLDEPL":
                    print("code:",c['code'],", testRun:",r,", value:",v['value'])
    except KeyError:
        continue

code: cb5be8c42b77476dbb5c4a23757bb0ed , testRun: 5fc7b922e268e5000a6f92ce , value: 68.31349857389378
code: cb5be8c42b77476dbb5c4a23757bb0ed , testRun: 5fc7b922e268e5000a6f92ce , value: 68.31349857389378
code: cb5be8c42b77476dbb5c4a23757bb0ed , testRun: 5fc7b922e268e5000a6f92ce , value: 68.31349857389378
code: ed521d104b139cfc7b305e5547a2cb23 , testRun: 5fc7b612e268e5000a6f9274 , value: 72.79347223726874
code: ed521d104b139cfc7b305e5547a2cb23 , testRun: 5fc7b4e6e268e5000a6f9249 , value: 72.79347223726874
code: ed521d104b139cfc7b305e5547a2cb23 , testRun: 5fc7b612e268e5000a6f9274 , value: 72.79347223726874
code: ed521d104b139cfc7b305e5547a2cb23 , testRun: 5fc7b4e6e268e5000a6f9249 , value: 72.79347223726874
code: ed521d104b139cfc7b305e5547a2cb23 , testRun: 5fc7b612e268e5000a6f9274 , value: 72.79347223726874
code: ed521d104b139cfc7b305e5547a2cb23 , testRun: 5fc7b4e6e268e5000a6f9249 , value: 72.79347223726874
